In [1]:
from sensory_data_client import create_data_client

client = create_data_client()

docs   = await client.list_doc()

endpoint='localhost:9000' accesskey='minioadmin' secretkey='minioadmin' bucket='default-bucket' secure=False


In [ ]:

lines  = await client.list_doclines()
objects = await client.list_stor(prefix="pdf/")